In [2]:
library(foreach)
library(doParallel)
library(Matching)
library(plyr)
library(tidyverse)
library(stringr)
library(magrittr)
library(caret)
library(itecv)
# detach("package:plyr", unload=TRUE)

In [3]:
loop_test = function(setup) {
    DGP = setup$DGP
    n_train = setup$n_train
    n_test = setup$n_test
    n_folds = setup$n_folds
    rep = setup$rep
    models = setup$models
    
    datas = setup_data(DGP$DGP, n_train, n_test, n_folds)
    estimates = datas %$% get_estimates(data, models, cv_index, test_index) 
    errors =  estimates %$% get_errors(cv_estimates, test_estimates, datas$aux_data) 
    true_selection_error = errors$true_selection_error %>%
        mutate(DGP=DGP$name, n_train=n_train, n_test=n_test, n_folds=n_folds, rep=rep)
    
    return(true_selection_error)
}

In [15]:
models = list(
# ranger_spec = list(method = "ranger",
#                    tune_grid = expand.grid(mtry = c(3,5),
#                                            splitrule="variance",
#                                            min.node.size=5)),
gbm_spec = list(method = "gbm",
                tune_grid = expand.grid(n.trees = seq(1,501,20), 
                                        interaction.depth=3, 
                                        shrinkage = 0.2, 
                                        n.minobsinnode=3)),
enet_spec = list(method = "enet",
                 tune_grid = expand.grid(fraction=0.5,
                                         lambda=exp(seq(-5,2,0.2))))
# nnet_spec = list(method = "nnet",
#                  tune_grid = expand.grid(size = seq(10,30,5),
#                                          decay = exp(seq(-7,0,1))))
)

simulation_params = list(DGP = powers_DGPs() %>% 
                             imap(~list(name=.y, DGP=.x)),
                         n_test = 2000,
                         n_train = 1000,
                         n_folds = c(2,10),
                         rep = 1:20) %>%
cross
setups = simulation_params %>%
    map(~c(.,"models"=list(models)))

In [8]:
# setups[[1]] %>% loop_test %>% data.frame

In [9]:
# results = setups %>% 
#     map(~loop_test(.)) %>%
#     bind_rows

In [ ]:
cl = makeCluster(4)
registerDoParallel(cl)
 
results = foreach(setup = setups, 
        .combine = bind_rows,
        .packages = (.packages()))  %dopar%  
loop_test(setup) 

stopCluster(cl)

In [ ]:
write_csv(results, "data/experimental_results_2.csv")